# Practical Lesson 10: Craftsman's Workshop

### __Problem Description__:

A craftsman's workshop has bars of 6 meters in length that need to be cut to obtain smaller bars in the following sizes: 

50 bars of 2 meters, 60 bars of 3 meters, and 90 bars of 4 meters. 

Develop an integer linear programming model that minimizes the waste from the cuts.


### Solution for Minimizing Wasted Space

In [56]:
from mip import *

def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value}\n")

    print("Solution:")
    for v in model.vars:
      if v.x > 0.00001 and v.name.find("x")!=-1:
        print(v.name, " = ", v.x)

def save(model, filename):
    model.write(filename) 
    with open(filename, "r") as f: 
        print(f.read())

In [57]:
qtd_patterns = 8
qtd_lengths = 3

# patterns[i] := (bars of length 2, bars of length 3, bars of length 4)
patterns = [
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [2, 0, 0],
    [0, 2, 0],
    [1, 1, 0],
    [1, 0, 1],
    [3, 0, 0]
]
# waste[i] := waste from cutting bar in pattern i
waste = [4, 3, 2, 2, 0, 1, 0, 0]

# demands[j] := demand for bars of type j
demands = [50, 60, 90]

In [58]:
model = Model(name="Cutting Stock Problem", sense=MINIMIZE, solver_name=CBC)

# x[i] := number of cuts for pattern i
x = [model.add_var(name=f"x_{i}", var_type=INTEGER, lb=0) for i in range(qtd_patterns)]

# Objective function
model.objective = xsum(x[i] * waste[i] for i in range(qtd_patterns))

# Constraints
for j in range(qtd_lengths):
    model.add_constr(name=f"bar_{j}", lin_expr=xsum(patterns[i][j] * x[i] for i in range(qtd_patterns)) == demands[j])

save(model, "../data/lesson10_1.lp")

\Problem name: Cutting Stock Problem

Minimize
OBJROW: 4 x_0 + 3 x_1 + 2 x_2 + 2 x_3 + x_5
Subject To
bar_0:  x_0 + 2 x_3 + x_5 + x_6 + 3 x_7 = 50
bar_1:  x_1 + 2 x_4 + x_5 = 60
bar_2:  x_2 + x_6 = 90
Bounds
Integers
x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 
End



In [59]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 80.0

Solution:
x_2  =  40.0
x_4  =  30.0
x_6  =  50.0


### Solution for Minimizing Patterns

Another solution could be minimizing the number of total pattern used.

For this, we can remove patterns that are subset of other.

In [60]:
qtd_patterns = 4
qtd_lengths = 3

# patterns[i] := (bars of length 2, bars of length 3, bars of length 4)
patterns = [
    [0, 2, 0],
    [1, 1, 0],
    [1, 0, 1],
    [3, 0, 0]
]
# waste[i] := waste from cutting bar in pattern i
waste = [0, 1, 0, 0]

# demands[j] := demand for bars of type j
demands = [50, 60, 90]

In [63]:
model = Model(name="Cutting Stock Problem", sense=MINIMIZE, solver_name=CBC)

# x[i] := number of cuts for pattern i
x = [model.add_var(name=f"x_{i}", var_type=INTEGER, lb=0) for i in range(qtd_patterns)]

# Objective function
model.objective = xsum(x[i] for i in range(qtd_patterns))

# Constraints
for j in range(qtd_lengths):
    model.add_constr(name=f"bar_{j}", lin_expr=xsum(patterns[i][j] * x[i] for i in range(qtd_patterns)) >= demands[j])

save(model, "../data/lesson10_2.lp")

\Problem name: Cutting Stock Problem

Minimize
OBJROW: x_0 + x_1 + x_2 + x_3
Subject To
bar_0:  x_1 + x_2 + 3 x_3 >= 50
bar_1:  2 x_0 + x_1 >= 60
bar_2:  x_2 >= 90
Bounds
Integers
x_0 x_1 x_2 x_3 
End



In [64]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 120.0

Solution:
x_0  =  30.0
x_2  =  90.0
